In [ ]:
# 计算从公园出发n分钟内的amentity数目

In [ ]:
#1. 加载公园数据

In [1]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx

# 选择适合的投影坐标系统
projected_crs = '3857'  # Web Mercator 投影

# 读取seattle 边界
# 加载seattle boundary
seattle_boundary = gpd.read_file("Seattle_City_Boundary.geojson")
seattle_boundary = seattle_boundary.to_crs(epsg=projected_crs)  
parks = gpd.read_file('SeattleParkPoints.geojson')
# 转换坐标系统CRS
parks_proj = parks.to_crs(epsg=projected_crs)  

In [2]:
# 加载公园细节数据
parks_detail = gpd.read_file('Park_Details.geojson')
parks_detail = parks_detail.rename(columns = {'PMA':'TOTAL_AREA','USE_':'TYPE'})
parks_detail = parks_detail.to_crs(epsg=projected_crs)  
parks_detail = parks_detail[parks_detail.is_valid]
parks_detail = gpd.clip(parks_detail,seattle_boundary)
parks_detail['centroid']=parks_detail.geometry.centroid

In [ ]:
#2. 加载amentity 数据

In [3]:
def ReadAmentityData(path,clip_mask,projected_crs):
    # 读取
    amentity_gdf = gpd.read_file(path)
    # 转换坐标系统
    if amentity_gdf.crs != projected_crs:
        amentity_gdf = amentity_gdf.to_crs(epsg=projected_crs)  
    # 裁剪
    amentity_gdf_clip = gpd.clip(amentity_gdf,clip_mask)
    return amentity_gdf_clip

In [5]:
# 1. 读取学校的数据
schools_gdf = ReadAmentityData(path = 'Seattle_Public_Schools_Sites.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
schools_gdf = schools_gdf[~schools_gdf['Status'].str.contains('Closed')]# not 
#schools_gdf.head()

,OBJECTID_1,OBJECTID,schID,schName,mapLabel,Status,esmshs,geometry
2,3,3,264,Rainier View,Rainier View,ELEM,ES,POINT (-13610274.057 6023884.827)
3,4,4,203,Arbor Heights,Arbor Heights,ELEM,ES,POINT (-13623010.760 6025670.631)
4,5,5,221,Emerson,Emerson,ELEM,ES,POINT (-13609769.121 6026514.487)
5,6,6,267,Roxhill,Roxhill,Option School with continuous enrollment,,POINT (-13622228.210 6027047.277)
9,10,10,215,Concord Int'l,Concord Int'l,ELEM,ES,POINT (-13617122.267 6027963.450)


In [7]:
# 2. 读取landmark的数据
landmarks_gdf = ReadAmentityData(path = 'Landmarks_and_Government_Buildings.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
#landmarks_gdf.head()

,OBJECTID,PERMANENT_IDENTIFIER,SOURCE_FEATUREID,SOURCE_DATASETID,SOURCE_DATADESC,SOURCE_ORIGINATOR,DATA_SECURITY,DISTRIBUTION_POLICY,LOADDATE,FTYPE,...,ADDRESSBUILDINGNAME,ADDRESS,CITY,STATE,ZIPCODE,GNIS_ID,FOOT_ID,COMPLEX_ID,GLOBALID,geometry
184874,1003832,{A4D79E6D-E90F-4E40-B933-5434EEA8FB65},None,2f97153f-59ba-442e-8b8e-c222ac07af34,780 TNMC Update 2/21/2023,U.S. Geological Survey,5.0,E4,"Tue, 21 Feb 2023 15:50:41 GMT",780,...,None,2721 Southwest Trenton Street,Seattle,WA,98126,2462269,None,None,5a46ac9d-5a03-46ba-a9af-a905d86c59f2,POINT (-13621845.468 6028039.808)
17019,1051652,{6E7174F1-8105-4D8E-8AA9-F6E12E7919CA},None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,5.0,E4,"Wed, 08 Mar 2023 21:53:12 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,1504959,None,None,566da1b2-9ab4-45ba-a686-4b0e4277a25a,POINT (-13621952.689 6030937.665)
171565,1471050,317a4f56-3ecd-4ca4-b73a-2eb43826af1f,None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,0.0,E4,"Wed, 08 Mar 2023 21:53:07 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,None,None,None,7c50bbdd-a30a-44df-a8c6-5b0541b235bc,POINT (-13622254.810 6031017.647)
171593,1471085,9957cb1b-a289-488f-ae3e-e9f2ffd50a9e,None,52740733-6aaf-4a55-967c-eb4358ba9a6d,820 TNMC Update 1/11/2024,U.S. Geological Survey,0.0,E4,"Thu, 11 Jan 2024 03:02:47 GMT",820,...,None,22nd Avenue South,Seattle,WA,98108,None,None,None,996c8467-6c8b-4d03-a2d3-59af5d9364b6,POINT (-13614950.916 6031818.468)
184858,1003815,{A71622B7-BAA4-48B1-8CC7-D8A1DD759EE5},None,86aaf0c3-7986-4d31-a3a5-e7154a52221d,780 TNMC Update 3/11/2023,U.S. Geological Survey,5.0,E4,"Sat, 11 Mar 2023 02:46:35 GMT",780,...,None,620 South Orcas Street,Seattle,WA,98108,2462221,None,None,1a953dfe-4bec-47fd-8360-eed9eeb67d63,POINT (-13617253.003 6032669.504)


In [9]:
# 3. 读取metro stop的数据
metro_stops_gdf = ReadAmentityData(path = 'Landmarks_and_Government_Buildings.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
#metro_stops_gdf.head()

,OBJECTID,PERMANENT_IDENTIFIER,SOURCE_FEATUREID,SOURCE_DATASETID,SOURCE_DATADESC,SOURCE_ORIGINATOR,DATA_SECURITY,DISTRIBUTION_POLICY,LOADDATE,FTYPE,...,ADDRESSBUILDINGNAME,ADDRESS,CITY,STATE,ZIPCODE,GNIS_ID,FOOT_ID,COMPLEX_ID,GLOBALID,geometry
184874,1003832,{A4D79E6D-E90F-4E40-B933-5434EEA8FB65},None,2f97153f-59ba-442e-8b8e-c222ac07af34,780 TNMC Update 2/21/2023,U.S. Geological Survey,5.0,E4,"Tue, 21 Feb 2023 15:50:41 GMT",780,...,None,2721 Southwest Trenton Street,Seattle,WA,98126,2462269,None,None,5a46ac9d-5a03-46ba-a9af-a905d86c59f2,POINT (-13621845.468 6028039.808)
17019,1051652,{6E7174F1-8105-4D8E-8AA9-F6E12E7919CA},None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,5.0,E4,"Wed, 08 Mar 2023 21:53:12 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,1504959,None,None,566da1b2-9ab4-45ba-a686-4b0e4277a25a,POINT (-13621952.689 6030937.665)
171565,1471050,317a4f56-3ecd-4ca4-b73a-2eb43826af1f,None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,0.0,E4,"Wed, 08 Mar 2023 21:53:07 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,None,None,None,7c50bbdd-a30a-44df-a8c6-5b0541b235bc,POINT (-13622254.810 6031017.647)
171593,1471085,9957cb1b-a289-488f-ae3e-e9f2ffd50a9e,None,52740733-6aaf-4a55-967c-eb4358ba9a6d,820 TNMC Update 1/11/2024,U.S. Geological Survey,0.0,E4,"Thu, 11 Jan 2024 03:02:47 GMT",820,...,None,22nd Avenue South,Seattle,WA,98108,None,None,None,996c8467-6c8b-4d03-a2d3-59af5d9364b6,POINT (-13614950.916 6031818.468)
184858,1003815,{A71622B7-BAA4-48B1-8CC7-D8A1DD759EE5},None,86aaf0c3-7986-4d31-a3a5-e7154a52221d,780 TNMC Update 3/11/2023,U.S. Geological Survey,5.0,E4,"Sat, 11 Mar 2023 02:46:35 GMT",780,...,None,620 South Orcas Street,Seattle,WA,98108,2462221,None,None,1a953dfe-4bec-47fd-8360-eed9eeb67d63,POINT (-13617253.003 6032669.504)


In [13]:
# 4. 读取housing的数据
housing_gdf = ReadAmentityData(path = 'King_County_Assessor_Residential_Unit_Types_and_Sizes.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
# 筛选集中住宅
housing_gdf = housing_gdf[housing_gdf['UNIT_TYPE'].str.contains('RES-2 Unit|RES-3 Unit|RES-4 Unit|Apartments', na=False)]
housing_gdf.rename(columns={'SQFTTOTLIVING:': 'square of living space'}, inplace=True)
#housing_gdf.head()

,OBJECTID,ADDRESS,MAJOR,PIN,BLDGNBR,PRES_USE,SECTION_USE,UNIT_TYPE,NBRTHISTYPE,BEDROOMS,...,VILLNUMB,UV_NAME,UC_NAME,CRA_NAME,TRBL20,POINT_X,POINT_Y,LAT,LON,geometry
131147,131148,11867 56TH PL S,957810,9578100140,1,Single Family(Res Use/Zone),Single-family Residence (351),SF-1 Unit,1,3,...,0.0,Outside Villages,None,Rainier Beach,11901.3010,1.285894e+06,184120.785427,47.495701,-122.267512,POINT (-13610755.565 6023362.855)
53914,53915,5364 S JUNIPER ST,334840,3348401099,1,Single Family(Res Use/Zone),Single-family Residence (351),SF-1 Unit,1,5,...,0.0,Outside Villages,None,Rainier Beach,11901.3008,1.286477e+06,184124.337585,47.495741,-122.265155,POINT (-13610493.151 6023369.361)
53910,53911,11855 55TH AVE S,334840,3348401095,1,Single Family(Res Use/Zone),Single-family Residence (351),SF-1 Unit,1,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3008,1.286606e+06,184127.305099,47.495755,-122.264632,POINT (-13610434.923 6023371.788)
131148,131149,11866 56TH PL S,957810,9578100150,1,Single Family(Res Use/Zone),Single-family Residence (351),SF-1 Unit,1,5,...,0.0,Outside Villages,None,Rainier Beach,11901.3008,1.286111e+06,184139.019971,47.495762,-122.266633,POINT (-13610657.721 6023372.921)
131146,131147,11859 56TH PL S,957810,9578100130,1,Single Family(Res Use/Zone),Single-family Residence (351),SF-1 Unit,1,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3010,1.285824e+06,184153.084247,47.495786,-122.267796,POINT (-13610787.178 6023376.854)


In [15]:
# 5. 读取工作机会的数据
jobs_gdf = ReadAmentityData(path = 'Business_Points_of_Interest_Seattle1.geojson',clip_mask = seattle_boundary,projected_crs = projected_crs)
jobs_gdf.rename(columns={'EMPNUM': 'Employers Number'}, inplace=True)
#jobs_gdf.head()

,OBJECTID,CONAME,ADDR,CITY,STATE_NAME,STATE,ZIP,ZIP4,NAICS,NAICS_ALL,...,SOURCE,ESRI_PID,DESC_,LATITUDE,LONGITUDE,CreationDate,Creator,EditDate,Editor,geometry
3747,3748,"Seattle Goodwill Industries, Inc",S Dearborn St,Seattle,Washington,WA,98144,,45951040,"45951040, 62419012",...,Data Axle,24741c12a5ff4e196a480cf655d2e78a,"CONAME, Seattle, Washington",47.596023,-122.323446,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13616983.721 6039908.677)
2529,2530,M C Electrical Vehicles,S Dearborn St,Seattle,Washington,WA,98144,2773,44111011,"44111011, 23821007, 81111410",...,Data Axle,02f83ac119f19ded293c785282c1fb85,"CONAME, Seattle, Washington",47.596059,-122.316273,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13616185.226 6039914.620)
3447,3448,Pacific Lighting Systems Inc,S Dearborn St,Seattle,Washington,WA,98144,2773,44912947,44912947,...,Data Axle,c62160bcb2bcde79132e6e7fcc403823,"CONAME, Seattle, Washington",47.596059,-122.316273,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13616185.226 6039914.620)
2927,2928,Public Storage,S Dearborn St,Seattle,Washington,WA,98144,2773,53113001,"53113001, 49311002, 49319002, 49319008, 531130...",...,Data Axle,54a0247f86b8557e824a62f9de540268,"CONAME, Seattle, Washington",47.596059,-122.316273,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13616185.226 6039914.620)
3763,3764,Seattle Information Tech Department,6th Ave S,Seattle,Washington,WA,98104,3826,92112006,92112006,...,Data Axle,7abe8ef5da970a1a85b643b740eb3abb,"CONAME, Seattle, Washington",47.596167,-122.326029,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13617271.259 6039932.448)


In [17]:
# 6. 读取MHA数据（affordable housing）
MHA_gdf= gpd.read_file('Seattle_Mandatory_Housing_Affordability_MHA_Zones.geojson')
# 转换坐标系统
if MHA_gdf.crs != projected_crs:
   MHA_gdf = MHA_gdf.to_crs(epsg=projected_crs)  
MHA_gdf =  MHA_gdf[ MHA_gdf.geometry.is_valid]
# 裁剪
MHA_gdf_clip = gpd.clip(MHA_gdf,seattle_boundary )
MHA_gdf_clip['centroid'] = MHA_gdf_clip.geometry.centroid
#MHA_gdf_clip.head()

,OBJECTID,ZONEID,ZONING,CONTRACT,ORDINANCE,EFFECTIVE,HISTORIC,PEDESTRIAN,SHORELINE,OVERLAY,...,PEDESTRIAN_PREV,SHORELINE_PREV,OVERLAY_PREV,LIGHTRAIL_PREV,BZONEID,PUBLIC_DESCRIPTION,CHAPTER,CHAPTER_LINK,geometry,centroid
671,672,5224,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13610055.007 6023559.091, -13610031...",POINT (-13609944.022 6023467.407)
691,692,5252,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608731.872 6024177.724, -13608806...",POINT (-13608748.347 6024221.130)
1678,1679,4958,LR3 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,2967.0,is a multifamily residential zone where reside...,Chapter 23.45,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13622848.750 6024534.624, -13622850...",POINT (-13622881.259 6024458.797)
1677,1678,4957,LR1 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,2966.0,is a multifamily residential zone where reside...,Chapter 23.45,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13611049.466 6025121.124, -13611047...",POINT (-13611012.153 6025175.461)
1379,1380,4708,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608554.849 6025919.535, -13608555...",POINT (-13608593.266 6025912.224)


In [ ]:
# 3. 计算xx分钟内可达的xx数目

In [ ]:
# 3.1 使用等时线矩阵

In [161]:
# 3.1.1 获取等时线矩阵
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import numpy as np
import time
import requests

#这里用mapbox来计算
def get_isochrone_mapbox(centroid, profile='walking', minutes=5,access_token=""):
    lon, lat = centroid.x, centroid.y
    url = f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/{lon},{lat}?contours_minutes={minutes}&access_token={access_token}"
    retries = 3
    while retries > 0:
        response = requests.get(url)
        if response.status_code == 200:#正常请求
            data = response.json()
            if 'features' in data and len(data['features']) > 0:
                coordinates = data['features'][0]['geometry']['coordinates']
                polygon = Polygon(coordinates)
                return polygon
            else:
                print(f"No features found for location: ({lon}, {lat})")
                return None
        elif response.status_code == 429:  # 请求太多
            print("Rate limit exceeded. Retrying...")
            time.sleep(60)  # 等待一分钟后重试
            retries -= 1
        else:
            print(f"Failed to fetch isochrone data for location: ({lon}, {lat})")
            return None
        time.sleep(12)  # 每次请求后等待12秒（相当于每分钟5次请求）
    return None


def CalculateTraveltimeOverlay(travel_time = 5,park_centroids = [],projected_crs = '3857',profile ='walking'):
    if(park_centroids == None):
        return None
    # 获取等时线
    accesstoken = 'pk.eyJ1Ijoic3liaWx3ZW4yMzMiLCJhIjoiY2x2cHBuMmtkMDRyOTJrbXh4eGhiOWR6cyJ9.1hNSzlQp8mLy37685Fo8Pg'
    isochrones = parks_centroids.apply(lambda centroid: get_isochrone_mapbox(centroid, profile = profile,minutes = travel_time, access_token=accesstoken))
    print(isochrones)
    isochrones_gdf = gpd.GeoDataFrame(geometry=isochrones)
    # 坐标转换
    isochrones_gdf.set_crs(epsg=4326, inplace=True,allow_override=True)
    # 在进行空间分析之前，确保所有涉及的 GeoDataFrame 使用相同的 CRS，一定要用wgs84。
    # 转换 CRS
    isochrones_gdf_proj = isochrones_gdf.to_crs(epsg=projected_crs)
    # 存储为 geojson 文件
    isochrones_gdf_proj.to_file(f'isochrones_gdf_proj_{profile}_{travel_time}.geojson', driver='GeoJSON')
    return isochrones_gdf_proj

In [199]:
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point

# 转换函数
def transform_point(_transformer,point):
    lon, lat = _transformer.transform(point.x, point.y)
    return Point(lon, lat)
# 获取中心点
parks_centroids = parks_detail.geometry.centroid
# 投影转换
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
parks_centroids_wgs84 = parks_centroids.apply(lambda point: transform_point(transformer, point))


In [209]:
def CalculateTraveltimeOverlay(travel_time = 5,projected_crs = '3857',profile ='walking'):
    # 获取等时线
    accesstoken = 'pk.eyJ1Ijoic3liaWx3ZW4yMzMiLCJhIjoiY2x2cHBuMmtkMDRyOTJrbXh4eGhiOWR6cyJ9.1hNSzlQp8mLy37685Fo8Pg'
    isochrones =[]
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    for index,row in parks_detail.iterrows():
        centroid = transform_point(transformer, row.geometry.centroid)
        isochrone = get_isochrone_mapbox(centroid, profile = profile,minutes = travel_time, access_token=accesstoken)
        isochrones.append(isochrone)
    isochrones_gdf  = gpd.GeoDataFrame({'geometry': isochrones})
    # 坐标转换
    isochrones_gdf.set_crs(epsg=4326, inplace=True,allow_override=True)
    # 在进行空间分析之前，确保所有涉及的 GeoDataFrame 使用相同的 CRS，一定要用wgs84。
    # 转换 CRS
    isochrones_gdf_proj = isochrones_gdf.to_crs(epsg=projected_crs)
    parks_detail['isochrones'] = isochrones_gdf_proj['geometry']
    parks_detail.set_geometry('geometry', inplace=True)
    return isochrones

Rate limit exceeded. Retrying...
Rate limit exceeded. Retrying...
Rate limit exceeded. Retrying...
Rate limit exceeded. Retrying...


In [ ]:
profile='walking'
travel_time = 10
isochrones = CalculateTraveltimeOverlay(travel_time = travel_time,projected_crs = '3857',profile = profile)
# 可以存取中途文件
parks_detail.to_csv(f'isochrones_gdf_park_deatails_{travel_time}_{profile}.csv', index=True)

In [177]:
# 创建 GeoDataFrame
isochrones_gdf  = gpd.GeoDataFrame({'centroid': parks_centroids, 'geometry': isochrones})
# 坐标转换
isochrones_gdf.set_crs(epsg=4326, inplace=True,allow_override=True)
# 在进行空间分析之前，确保所有涉及的 GeoDataFrame 使用相同的 CRS，一定要用wgs84。
# 转换 CRS
isochrones_gdf_proj = isochrones_gdf.to_crs(epsg=projected_crs)
isochrones_gdf_proj.set_geometry('geometry', inplace=True)

In [25]:
# 3.1.2 计算xx时间范围内xx的数目

def CalculateAmentityCountInIsochrones(amentity_gdf_clip, amentity_name, intersect_grid):
    # 执行空间连接，找到每个等时线范围内的设施
    amentity_within_isochrones = gpd.sjoin(amentity_gdf_clip, intersect_grid, how='inner', predicate='within')
    
    # 统计每个等时线范围内的设施数目
    amentity_counts = amentity_within_isochrones.groupby('index_right').size()
    
    # 创建一个新的列，填充统计结果
    amentity_index = f'{amentity_name}_count'
    intersect_grid[amentity_index] = intersect_grid.index.map(amentity_counts).fillna(0).astype(int)

    return intersect_grid


In [183]:
parks_detail = CalculateAmentityCountInIsochrones(amentity_gdf_clip = landmarks_gdf,amentity_name = 'Landmarks',intersect_grid = parks_detail)
parks_detail.head()

,centroid,geometry,Landmarks_count
0,POINT (-13626367.925 6036918.326),"POLYGON ((-13625254.759 6038246.193, -13625366...",0
1,POINT (-13619819.517 6049761.138),"POLYGON ((-13619596.835 6051299.652, -13619930...",0
2,POINT (-13624677.828 6052567.874),"POLYGON ((-13624566.471 6054058.516, -13624677...",1
3,POINT (-13624049.657 6047882.884),"POLYGON ((-13623938.295 6049868.161, -13624088...",1
4,POINT (-13621080.948 6045338.439),"POLYGON ((-13621192.265 6047017.952, -13621414...",2


In [185]:
parks_detail = CalculateAmentityCountInIsochrones(amentity_gdf_clip = schools_gdf,amentity_name = 'Schools',intersect_grid = parks_detail)
parks_detail.head()

,centroid,geometry,Landmarks_count,Schools_count
0,POINT (-13626367.925 6036918.326),"POLYGON ((-13625254.759 6038246.193, -13625366...",0,1
1,POINT (-13619819.517 6049761.138),"POLYGON ((-13619596.835 6051299.652, -13619930...",0,2
2,POINT (-13624677.828 6052567.874),"POLYGON ((-13624566.471 6054058.516, -13624677...",1,2
3,POINT (-13624049.657 6047882.884),"POLYGON ((-13623938.295 6049868.161, -13624088...",1,1
4,POINT (-13621080.948 6045338.439),"POLYGON ((-13621192.265 6047017.952, -13621414...",2,5


In [187]:
parks_detail = CalculateAmentityCountInIsochrones(amentity_gdf_clip = metro_stops_gdf,amentity_name = 'Metros',intersect_grid = parks_detail)
parks_detail.head()

,centroid,polygon,Landmarks_count,Schools_count,Metros_count,Housings_count
0,POINT (-122.4077457365096 47.57790484908049),"POLYGON ((-13625254.759 6038246.193, -13625366...",0,1,0,1662
1,POINT (-122.3489203869725 47.65567341516601),"POLYGON ((-13619596.835 6051299.652, -13619930...",0,2,0,4341
2,POINT (-122.3925633373464 47.67265396954006),"POLYGON ((-13624566.471 6054058.516, -13624677...",1,2,1,4419
3,POINT (-122.3869203841367 47.64430702796048),"POLYGON ((-13623938.295 6049868.161, -13624088...",1,1,1,2611
4,POINT (-122.3602520179865 47.62890519351506),"POLYGON ((-13621192.265 6047017.952, -13621414...",2,5,2,2939


In [189]:
parks_detail = CalculateAmentityCountInIsochrones(amentity_gdf_clip = housing_gdf,amentity_name = 'Housings',intersect_grid = parks_detail)
parks_detail.head()

,centroid,polygon,Landmarks_count,Schools_count,Metros_count,Housings_count
0,POINT (-122.4077457365096 47.57790484908049),"POLYGON ((-13625254.759 6038246.193, -13625366...",0,1,0,1662
1,POINT (-122.3489203869725 47.65567341516601),"POLYGON ((-13619596.835 6051299.652, -13619930...",0,2,0,4341
2,POINT (-122.3925633373464 47.67265396954006),"POLYGON ((-13624566.471 6054058.516, -13624677...",1,2,1,4419
3,POINT (-122.3869203841367 47.64430702796048),"POLYGON ((-13623938.295 6049868.161, -13624088...",1,1,1,2611
4,POINT (-122.3602520179865 47.62890519351506),"POLYGON ((-13621192.265 6047017.952, -13621414...",2,5,2,2939


In [191]:
# 因为每个工作点 对应了多个工作机会。
# 执行空间连接
jobs_within_isochrones = gpd.sjoin(jobs_gdf, parks_detail, how='inner', predicate='within')

# 计算每个区域内 Employers Number 的总和
aggregated_employers = jobs_within_isochrones.groupby('index_right')['Employers Number'].sum().reset_index()

# 将总和信息添加到 parks_detail
parks_detail = parks_detail.merge(aggregated_employers, left_index=True, right_on='index_right', how='left')

# 重命名新列
parks_detail = parks_detail.rename(columns={'Employers Number': 'job_count'})

# 填充缺失值，并确保列的类型为整数
parks_detail['job_count'] = parks_detail['job_count'].fillna(0).astype(int)

# 删掉index_right这一列
parks_detail = parks_detail.drop(columns=['index_right'])
parks_detail.head()

,centroid,polygon,Landmarks_count,Schools_count,Metros_count,Housings_count,job_count
NaN,POINT (-122.4077457365096 47.57790484908049),"POLYGON ((-13625254.759 6038246.193, -13625366...",0,1,0,1662,0
NaN,POINT (-122.3489203869725 47.65567341516601),"POLYGON ((-13619596.835 6051299.652, -13619930...",0,2,0,4341,0
NaN,POINT (-122.3925633373464 47.67265396954006),"POLYGON ((-13624566.471 6054058.516, -13624677...",1,2,1,4419,0
NaN,POINT (-122.3869203841367 47.64430702796048),"POLYGON ((-13623938.295 6049868.161, -13624088...",1,1,1,2611,0
NaN,POINT (-122.3602520179865 47.62890519351506),"POLYGON ((-13621192.265 6047017.952, -13621414...",2,5,2,2939,0


In [193]:
parks_detail = CalculateAmentityCountInIsochrones(amentity_gdf_clip = MHA_gdf_clip,amentity_name = 'Affordable_Housings',intersect_grid = parks_detail)
parks_detail.head()

,centroid,polygon,Landmarks_count,Schools_count,Metros_count,Housings_count,job_count,Affordable_Housings_count
NaN,POINT (-122.4077457365096 47.57790484908049),"POLYGON ((-13625254.759 6038246.193, -13625366...",0,1,0,1662,0,0
NaN,POINT (-122.3489203869725 47.65567341516601),"POLYGON ((-13619596.835 6051299.652, -13619930...",0,2,0,4341,0,0
NaN,POINT (-122.3925633373464 47.67265396954006),"POLYGON ((-13624566.471 6054058.516, -13624677...",1,2,1,4419,0,0
NaN,POINT (-122.3869203841367 47.64430702796048),"POLYGON ((-13623938.295 6049868.161, -13624088...",1,1,1,2611,0,0
NaN,POINT (-122.3602520179865 47.62890519351506),"POLYGON ((-13621192.265 6047017.952, -13621414...",2,5,2,2939,0,0


In [283]:
#parks_detail.to_csv(f'isochrones_gdf_parks_proj_{profile}_{travel_time}_with_Centroids.csv')

In [ ]:
# 3.2 使用travel_time_matrix矩阵

In [219]:
import r5py
import geopandas
import datetime
import time
from tqdm import tqdm

# 读取pbd文件
osm_pbf_path = "seattle.osm.pbf"
    
# 创建 TransportNetwork 对象
transport_network = r5py.TransportNetwork(
    osm_pbf_path
)




In [223]:
def CalculateTravelTimeMatrix(parks_proj,amentity_proj,amentity_name):
    start_time = time.time()
   
    # 唯一标识符的列分别重命名为 'id'
    parks_4timematrix = parks_proj.rename(columns={"OBJECTID": "id"})  # 替换为数据中的唯一标识符列名
    amentity_4timematrix = amentity_proj.rename(columns={"OBJECTID": "id"})  # 替换为数据中的唯一标识符列名
    
    # 获取网格单元的中心点作为起点和终点
    origins = parks_4timematrix.copy()
    origins["geometry"] = origins.geometry.centroid
    
    destinations = amentity_4timematrix.copy()
    destinations["geometry"] = destinations.geometry.centroid

    # 定义批次大小
    batch_size_origins = len(origins) // 3 # 可以根据实际情况调整 
    batch_size_destinations = len(destinations) // 3 # 可以根据实际情况调整 
    print(f'number of destination is :{batch_size_destinations}')
    
    # 将起点和终点分成批次
    origins_batches = np.array_split(origins, len(origins) // batch_size_origins + 1)
    destinations_batches = np.array_split(destinations, len(destinations) // batch_size_destinations + 1)
    
    all_travel_times = []
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"开始计算运行时间: {elapsed_time} 秒")
    
    # 逐批次计算旅行时间
    for origins_batch in tqdm(origins_batches):
        for destinations_batch in tqdm(destinations_batches):
            travel_time_matrix = r5py.TravelTimeMatrixComputer(
                transport_network,
                origins=origins_batch,
                destinations=destinations_batch,
                transport_modes=[r5py.TransportMode.BICYCLE],
                departure=datetime.datetime(2024, 8, 3, 9, 0, 0),
            ).compute_travel_times()
            all_travel_times.append(travel_time_matrix)
    
    # 合并所有批次的结果
    final_travel_time_matrix = pd.concat(all_travel_times, ignore_index=True)
    
    # 保存为 CSV 文件
    file_name = f'travel_time_matrix_{amentity_name}.csv'
    final_travel_time_matrix.to_csv(file_name, index=False)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"计算运行时间: {elapsed_time} 秒")
    return final_travel_time_matrix 


In [ ]:
travel_10_school = CalculateTravelTimeMatrix(parks_proj,schools_gdf,'school')

In [233]:
import pandas as pd
import geopandas as gpd

# 读取旅行时间矩阵
travel_time_matrix = pd.read_csv('travel_time_matrix_schools.csv')

# 设置时间阈值
time_threshold = 10  # 10分钟

# 过滤符合时间阈值的记录
filtered_travel_time_matrix = travel_time_matrix[travel_time_matrix["travel_time"] <= time_threshold]

# 计算每个from_id可以到达的唯一to_id数量
accessible_schools_count = filtered_travel_time_matrix.groupby("from_id").agg({'to_id': 'nunique'}).reset_index()
accessible_schools_count.columns = ['from_id', 'accessible_schools_count']

# 确保 parks_proj 包含 'from_id' 列
parks_proj['from_id'] = parks_proj['OBJECTID']

# 合并数据，将 'accessible_schools_count' 列添加到 parks_proj
# 将 parks_proj 转换为 DataFrame 进行合并
parks_proj_df = pd.DataFrame(parks_proj)

# 合并数据，使用 'from_id' 进行匹配
parks_proj_merged = parks_proj_df.merge(accessible_schools_count, on='from_id', how='left')

# 将合并后的 DataFrame 转换为 GeoDataFrame 并设置 geometry 列
parks_proj_gdf = gpd.GeoDataFrame(parks_proj_merged, geometry='geometry')
print(parks_proj_gdf)


     OBJECTID                PROPNAME                    ADDRESS DIVISION  \
0        7044         Alki Playground          5817 SW Lander St    South   
1        7045     B.F. Day Playground         4020 Fremont Ave N    North   
2        7046      Ballard Playground            2644 NW 60th St    North   
3        7047      Bayview Playground            2614 24th Ave W  Central   
4        7048    Bayview-Kinnear Park  3rd Ave W / W Prospect St  Central   
..        ...                     ...                        ...      ...   
150      7194     Golden Gardens Park         8498 Seaview Pl NW    North   
151      7195    Ballard Commons Park           5701 22nd Ave NW     None   
152      7196  Duwamish Waterway Park            7900 10th Ave S     None   
153      7197           Lake City HUB          12510 33rd Ave NE     None   
154      7198       Occidental Square        117 S Washington St     None   

      ADA                     GIS_EDT_DT   LATITUDE   LONGITUDE       POINT

In [311]:
# 计算从公园出发到最近的xx的距离

# 1. 筛选我们有的数据点（只选在15分钟范围内的）
# 2. 计算距离

# 计算每个公园到最近amentity的距离（米）
def calculate_nearest_school_distance(parks, amentity_within_isochrones):
    distances = []
    for _, park in parks.iterrows():
        park_point = park.geometry.centroid
        # 计算公园到筛选后amentity的距离
        distances_to_amentity = amentity_within_isochrones.geometry.distance(park_point)
        # 找到最小距离
        nearest_distance = distances_to_amentity.min() if not distances_to_amentity.empty else float('inf')
        distances.append(nearest_distance)
    return distances
    

In [305]:
parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)
parks_detail['centroid'] = parks_detail['centroid'].apply(wkt.loads)
# Create GeoDataFrame
selected_columns = parks_detail[['OBJECTID', 'isochrones','centroid']]
parks_isochrones = gpd.GeoDataFrame(selected_columns, geometry='isochrones')
parks_isochrones.set_crs(epsg=projected_crs,inplace=True)
parks_isochrones = parks_isochrones.to_crs(epsg=projected_crs)
parks_isochrones.head()

,OBJECTID,isochrones,centroid
NaN,660345,"POLYGON ((-13610500.000 6027250.000, -13611100...",POINT (-13622700 6023630)
NaN,659362,"POLYGON ((-13619400.000 6042830.000, -13619600...",POINT (-13622800 6023680)
NaN,659360,"POLYGON ((-13622000.000 6059120.000, -13622100...",POINT (-13622800 6023730)
NaN,660174,"POLYGON ((-13613200.000 6037070.000, -13613700...",POINT (-13622900 6023750)
NaN,659359,"POLYGON ((-13615000.000 6041910.000, -13615100...",POINT (-13622900 6023770)


In [330]:
amentity_within_isochrones = gpd.sjoin(metro_stops_gdf, parks_isochrones, how='inner', predicate='within')
parks_isochrones['nearest_metro_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,nearest_metro_distance,nearest_school_distance,nearest_landmark_distance
NaN,660345,"POLYGON ((-13610500.000 6027250.000, -13611100...",POINT (-13622700 6023630),4488.539813,3444.342786,4488.539813
NaN,659362,"POLYGON ((-13619400.000 6042830.000, -13619600...",POINT (-13622800 6023680),4454.809485,3406.163533,4454.809485
NaN,659360,"POLYGON ((-13622000.000 6059120.000, -13622100...",POINT (-13622800 6023730),4339.999976,3294.326229,4339.999976
NaN,660174,"POLYGON ((-13613200.000 6037070.000, -13613700...",POINT (-13622900 6023750),4362.209254,3311.548279,4362.209254
NaN,659359,"POLYGON ((-13615000.000 6041910.000, -13615100...",POINT (-13622900 6023770),4359.353641,3306.170624,4359.353641


In [318]:
parks_isochrones.head()

,OBJECTID,isochrones,centroid,nearest_metro_distance,nearest_school_distance,nearest_landmark_distance
NaN,660345,"POLYGON ((-13610500.000 6027250.000, -13611100...",POINT (-13622700 6023630),4488.539813,3444.342786,4488.539813
NaN,659362,"POLYGON ((-13619400.000 6042830.000, -13619600...",POINT (-13622800 6023680),4454.809485,3406.163533,4454.809485
NaN,659360,"POLYGON ((-13622000.000 6059120.000, -13622100...",POINT (-13622800 6023730),4339.999976,3294.326229,4339.999976
NaN,660174,"POLYGON ((-13613200.000 6037070.000, -13613700...",POINT (-13622900 6023750),4362.209254,3311.548279,4362.209254
NaN,659359,"POLYGON ((-13615000.000 6041910.000, -13615100...",POINT (-13622900 6023770),4359.353641,3306.170624,4359.353641


In [ ]:
amentity_within_isochrones = gpd.sjoin(schools_gdf, parks_isochrones, how='inner', predicate='within')
parks_isochrones['nearest_school_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)

In [ ]:
amentity_within_isochrones = gpd.sjoin(landmarks_gdf, parks_isochrones, how='inner', predicate='within')
parks_isochrones['nearest_landmark_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)

In [320]:
amentity_within_isochrones = gpd.sjoin(housing_gdf, parks_isochrones, how='inner', predicate='within')
parks_isochrones['nearest_residential_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)



KeyboardInterrupt



In [332]:
amentity_within_isochrones = gpd.sjoin(MHA_gdf_clip, parks_isochrones, how='inner', predicate='within')
parks_isochrones['nearest_MHA_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)

parks_isochrones.head()

Exception ignored in: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/8q/tr91_j755cn3r5tsj18hdjnr0000gn/T/ipykernel_50369/2290693236.py", line 2, in <module>
    parks_isochrones['nearest_MHA_distance'] = calculate_nearest_amentity_distance(parks_detail, amentity_within_isochrones)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/8q/tr91_j755cn3r5tsj18hdjnr0000gn/T/ipykernel_50369/3422761735.py", line 12, in calculate_nearest_amentity_distance
    distances_to_amentity = amentity_within_isochrones.geometry.distance(park_point)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/geopandas/base.py", line 2577, in distance
    return _binary

KeyboardInterrupt: 

In [338]:
parks_isochrones.to_csv('parks_isochrones.csv',index=True)

In [261]:
amentity_within_isochrones = gpd.sjoin(metro_stops_gdf, intersect_grid_15, how='inner', predicate='within')
parks_proj['nearest_metro_distance'] = calculate_nearest_school_distance(parks_proj, amentity_within_isochrones)
parks_proj

,OBJECTID,PROPNAME,ADDRESS,DIVISION,ADA,GIS_EDT_DT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,...,AMWO_ID,RES1,RES2,GLOBALID,GIS_CREATOR,GIS_CRT_DT,GIS_EDITOR,SDQL,geometry,nearest_metro_distance
0,7044,Alki Playground,5817 SW Lander St,South,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.577899,-122.407731,1.251850e+06,214775.000001,...,PLAY-ALKIPG,None,None,17ca3d9c-c23f-41f4-927f-5f4583822a15,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13626367.925 6036918.326),2432.469299
1,7045,B.F. Day Playground,4020 Fremont Ave N,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.655668,-122.348906,1.266925e+06,242845.296892,...,PLAY-DAYPG,None,None,2b87dea1-038d-4a59-98f0-db9ad09725ff,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13619819.517 6049761.138),1427.100127
2,7046,Ballard Playground,2644 NW 60th St,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.672648,-122.392548,1.256297e+06,249252.812470,...,PLAY-BLPG,None,None,ff837c9b-46fd-4b79-8eda-d51a40268425,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13624677.828 6052567.874),1621.959196
3,7047,Bayview Playground,2614 24th Ave W,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.644301,-122.386906,1.257478e+06,238886.812472,...,PLAY-BVPG,None,None,c7a03c40-dbb7-4dcf-ac40-bb26e9c9b3b2,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13624049.657 6047882.884),1271.943017
4,7048,Bayview-Kinnear Park,3rd Ave W / W Prospect St,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.628900,-122.360237,1.263939e+06,233138.296906,...,PLAY-KERPK,None,None,10ab7d68-dd5c-43cc-98a4-b85c233bdc56,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13621080.948 6045338.439),772.456642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,7194,Golden Gardens Park,8498 Seaview Pl NW,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.691781,-122.403738,1.253683e+06,256286.406237,...,PLAY-GGPK,None,None,ecc5f2c2-c26d-4ce9-9f7e-57d26af9d709,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13625923.473 6055731.451),3754.741024
151,7195,Ballard Commons Park,5701 22nd Ave NW,None,None,"Mon, 29 Jul 2024 19:16:43 GMT",NaN,NaN,NaN,NaN,...,PLAY-BLSKATPK,None,None,64b01114-b766-48a0-b592-4f7281ddc6c9,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13623839.608 6052228.416),740.629421
152,7196,Duwamish Waterway Park,7900 10th Ave S,None,None,"Mon, 29 Jul 2024 19:16:43 GMT",NaN,NaN,NaN,NaN,...,PLAY-DUWPK,None,None,f5abc4fc-fd79-4f52-8330-eadf29c6f1e1,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13616563.630 6029146.786),3589.537057
153,7197,Lake City HUB,12510 33rd Ave NE,None,None,"Mon, 29 Jul 2024 19:16:43 GMT",NaN,NaN,NaN,NaN,...,PLAY-LKCYHUB,None,None,dd0058be-e656-4f39-b3a3-e4956f5dca38,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13613536.957 6060388.808),349.826320


In [277]:
def calculate_nearest_park_distance(parks):
    # 列表存储每个公园到最近公园的距离
    distances = []
    
    # 对每个公园进行处理
    for _, park in parks.iterrows():
        park_point = park.geometry.centroid
        # 计算该公园到所有其他公园的距离
        distances_to_other_parks = parks.loc[parks.index != _].geometry.distance(park_point)
        # 找到最小距离
        nearest_distance = distances_to_other_parks.min() if not distances_to_other_parks.empty else float('inf')
        distances.append(nearest_distance)
    
    return distances

In [340]:
# 计算每个公园到最近公园的距离
nearest_park_distances = calculate_nearest_park_distance(parks_detail)
parks_detail['nearest_park_distance'] = nearest_park_distances
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,centroid,isochrones,geometry,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count,job_count,nearest_park_distance
NaN,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,POINT (-13622700 6023630),"POLYGON ((-13610500 6027250, -13611100 6027020...","POLYGON ((-13622741.737 6023640.678, -13622737...",0,0,0,0,0,0,0.0
NaN,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,POINT (-13622800 6023680),"POLYGON ((-13619400 6042830, -13619600 6042900...","POLYGON ((-13622800.051 6023678.899, -13622809...",0,0,0,0,0,0,0.0
NaN,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,POINT (-13622800 6023730),"POLYGON ((-13622000 6059120, -13622100 6059030...","POLYGON ((-13622845.404 6023793.752, -13622749...",0,0,0,0,0,0,0.0
NaN,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,POINT (-13622900 6023750),"POLYGON ((-13613200 6037070, -13613700 6037050...","POLYGON ((-13622858.724 6023793.883, -13622845...",0,0,0,0,0,0,0.0
NaN,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,POINT (-13622900 6023770),"POLYGON ((-13615000 6041910, -13615100 6041910...","POLYGON ((-13622948.679 6023814.258, -13622922...",0,0,0,0,0,0,0.0


In [328]:
parks_detail.columns

Index(['OBJECTID', 'NAME', 'PMA_NAME', 'ADDRESS', 'PIN', 'SUBPARCEL',
       'TOTAL_AREA', 'OWNER', 'LEASE', 'MAINT', 'TYPE', 'ACQ_DATE', 'NAMEFLAG',
       'REVIEW_DATE', 'AMWOID', 'SDQL', 'SE_ANNO_CAD_DATA', 'GIS_CRT_DT',
       'GIS_EDT_DT', 'GlobalID', 'centroid', 'isochrones', 'geometry',
       'Schools_count', 'Landmarks_count', 'Metros_count', 'Housings_count',
       'Affordable_Housings_count', 'job_count', 'nearest_park_distance'],
      dtype='object')

In [279]:
nearest_park_distances = calculate_nearest_park_distance(parks_proj)
parks_proj['nearest_park_distance'] = nearest_park_distances

In [351]:
print(parks_detail.head())
parks_detail.to_csv("parks_detail_accessibility_10.csv", index=True)

     OBJECTID        NAME    PMA_NAME ADDRESS         PIN  SUBPARCEL  \
NaN    660345  SEOLA PARK  Seola Park          7110000066    22024.0   
NaN    659362  SEOLA PARK  Seola Park          1123039030     9627.0   
NaN    659360  SEOLA PARK  Seola Park          1223039013     9627.0   
NaN    660174  SEOLA PARK  Seola Park    None        None     9627.0   
NaN    659359  SEOLA PARK  Seola Park          1123039028     9628.0   

     TOTAL_AREA OWNER LEASE MAINT  ...                   centroid  \
NaN         466   DPR     N   DPR  ...  POINT (-13622700 6023630)   
NaN         466   DPR     N   DPR  ...  POINT (-13622800 6023680)   
NaN         466   DPR     N   DPR  ...  POINT (-13622800 6023730)   
NaN         466   DPR     N   DPR  ...  POINT (-13622900 6023750)   
NaN         466   DPR     N   DPR  ...  POINT (-13622900 6023770)   

                                            isochrones  \
NaN  POLYGON ((-13610500 6027250, -13611100 6027020...   
NaN  POLYGON ((-13619400 6042830, -1

In [342]:
# 计算每个公园到最近的自行车网络的距离
# 1. 加载自行车网络
import numpy as np
import osmnx as ox

# 使用 osmnx 库从 GraphML 文件加载自行车网络数据。
bike_network = ox.load_graphml('bike_network.graphml')
nodes, edges = ox.graph_to_gdfs(bike_network)

# 重置索引，将 'u' 和 'v' 转化为普通列
edges = edges.reset_index()


In [344]:
# 3. 计算每个公园到所有自行车网络边的距离
def calculate_nearest_distance(point, edges):
    # 计算公园到所有自行车网络边的距离
    distances = [point.distance(edge) for edge in edges.geometry]
    return min(distances) if distances else float('inf')

# 1. 筛选时间范围内的自行车网络边
edges = edges.to_crs(epsg=projected_crs)
#edges_within_isochrones = gpd.sjoin(edges, intersect_grid_15, how='inner', predicate='intersects')

#edges_within_isochrones

In [346]:
edges = edges.to_crs(epsg=projected_crs)
edges_within_isochrones = gpd.sjoin(edges, parks_isochrones, how='inner', predicate='intersects')


In [348]:
parks_detail['distance_to_nearest_bike_network'] = parks_detail.geometry.apply(lambda x: calculate_nearest_distance(x, edges_within_isochrones))
parks_detail.to_file("parks_detail_accessibility_10.geojson", driver='GeoJSON')


KeyboardInterrupt



In [283]:
# 添加一个新列来存储距离
parks_proj['distance_to_nearest_bike_network'] = parks_proj.geometry.apply(lambda x: calculate_nearest_distance(x, edges_within_isochrones))

# 将距离转换为英尺（原始数据是以米为单位）
#parks_proj['distance_to_nearest_bike_network_ft'] = parks_proj['distance_to_nearest_bike_network'] * 3.28084
parks_proj.head()

,OBJECTID,PROPNAME,ADDRESS,DIVISION,ADA,GIS_EDT_DT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,...,RES2,GLOBALID,GIS_CREATOR,GIS_CRT_DT,GIS_EDITOR,SDQL,geometry,nearest_metro_distance,nearest_park_distance,distance_to_nearest_bike_network
0,7044,Alki Playground,5817 SW Lander St,South,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.577899,-122.407731,1.251850e+06,214775.000001,...,None,17ca3d9c-c23f-41f4-927f-5f4583822a15,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13626367.925 6036918.326),2432.469299,243.134381,34.089434
1,7045,B.F. Day Playground,4020 Fremont Ave N,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.655668,-122.348906,1.266925e+06,242845.296892,...,None,2b87dea1-038d-4a59-98f0-db9ad09725ff,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13619819.517 6049761.138),1427.100127,1431.962352,39.717508
2,7046,Ballard Playground,2644 NW 60th St,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.672648,-122.392548,1.256297e+06,249252.812470,...,None,ff837c9b-46fd-4b79-8eda-d51a40268425,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13624677.828 6052567.874),1621.959196,904.347441,43.827003
3,7047,Bayview Playground,2614 24th Ave W,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.644301,-122.386906,1.257478e+06,238886.812472,...,None,c7a03c40-dbb7-4dcf-ac40-bb26e9c9b3b2,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13624049.657 6047882.884),1271.943017,772.891742,61.165600
4,7048,Bayview-Kinnear Park,3rd Ave W / W Prospect St,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.628900,-122.360237,1.263939e+06,233138.296906,...,None,10ab7d68-dd5c-43cc-98a4-b85c233bdc56,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (-13621080.948 6045338.439),772.456642,788.296098,27.899002


In [287]:
parks_proj.to_file("distance_accessibility_15.geojson", driver='GeoJSON')